Notes:

Model: the code uses Ollama (llama3 by default). Pull it once: ollama pull llama3.

RAG: uses lightweight TF-IDF (scikit-learn)

Outputs: saved to outputs/summary.json, outputs/suggestions.json, and optional batch artifacts.

In [ ]:
# (Optional) Install light dependencies

# If you already have these, you can skip.
# Keeps it light: NO pyarrow, NO Chroma, NO transformers.
!pip install -q pandas==2.2.2 scipy==1.11.4 scikit-learn==1.5.2 langchain==0.2.10 langchain-core==0.2.10 langchain-ollama==0.1.0 pydantic==2.7.0


In [1]:
# Imports, paths and helpers

import os, json, math, re
from pathlib import Path
from dataclasses import dataclass
import pandas as pd
from typing import Dict, Any, List, Optional, Tuple

# Project dirs
ROOT = Path.cwd()
DATA = ROOT / "data"
RAGDIR = ROOT / "rag"
OUT = ROOT / "outputs"
OUT.mkdir(exist_ok=True, parents=True)
RAGDIR.mkdir(exist_ok=True, parents=True)

print("ROOT:", ROOT)
print("DATA:", DATA)
print("RAG:", RAGDIR)
print("OUT:", OUT)




/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


ROOT: /Users/abhinavsharma/Documents/copilot_mvp
DATA: /Users/abhinavsharma/Documents/copilot_mvp/data
RAG: /Users/abhinavsharma/Documents/copilot_mvp/rag
OUT: /Users/abhinavsharma/Documents/copilot_mvp/outputs


In [2]:
# Stats: z-test for proportions + verdict rules

from math import erf, sqrt

@dataclass
class ZTestResult:
    control_rate: float
    treatment_rate: float
    lift: float
    p_value: float
    ci_low: float
    ci_high: float

def _phi(z):  # CDF of standard normal
    return 0.5 * (1.0 + erf(z / sqrt(2.0)))

def _z_from_alpha_two_tailed(alpha: float) -> float:
    # binary search inverse CDF
    lo, hi = -10.0, 10.0
    target = 1 - alpha/2
    for _ in range(80):
        mid = (lo + hi) / 2
        if _phi(mid) < target: lo = mid
        else: hi = mid
    return (lo + hi) / 2

def z_test_proportions(n1:int, x1:int, n2:int, x2:int, alpha=0.05) -> ZTestResult:
    p1 = x1 / max(n1, 1)
    p2 = x2 / max(n2, 1)
    lift = (p2 - p1)
    # pooled std error
    p_pool = (x1 + x2) / max(n1 + n2, 1)
    se = sqrt(p_pool*(1-p_pool)*(1/n1 + 1/n2)) if n1>0 and n2>0 else 1e9
    z = 0 if se==0 else (p2 - p1) / se
    pval = 2 * (1 - _phi(abs(z)))
    # CI around difference
    zcrit = _z_from_alpha_two_tailed(alpha)
    ci_low, ci_high = lift - zcrit*se, lift + zcrit*se
    return ZTestResult(p1, p2, lift, pval, ci_low, ci_high)

def verdict_from_result(res: ZTestResult, alpha=0.05, min_abs_lift=0.0) -> str:
    if res.p_value < alpha and abs(res.lift) > min_abs_lift:
        return "Win" if res.lift > 0 else "Lose"
    return "Inconclusive"



In [3]:
# CSV loader (expects: variant, n, conversions, [experiemnt_name, hypothesis, primary_metric])

@dataclass
class Experiment:
    name: str
    hypothesis: str
    primary_metric: str
    rows: List[Dict[str, Any]]

def load_experiment_csv(csv_path: str) -> Experiment:
    df = pd.read_csv(csv_path)
    req = {"variant","n","conversions"}
    assert req.issubset(set(df.columns)), f"CSV must have columns: {req}"
    # best-effort meta
    name = (df["experiment_name"].iloc[0] if "experiment_name" in df.columns else Path(csv_path).stem)
    hypothesis = (df["hypothesis"].iloc[0] if "hypothesis" in df.columns else "N/A")
    primary = (df["primary_metric"].iloc[0] if "primary_metric" in df.columns else "conversion_rate")

    # Normalise variants to control/treatment order
    variants = list(df["variant"].astype(str).str.lower().unique())
    # heuristic: control first if present
    if "control" in variants:
        order = ["control"] + [v for v in variants if v != "control"]
    else:
        order = variants
    rows = []
    for v in order:
        sub = df[df["variant"].astype(str).str.lower() == v].iloc[0]
        rows.append({"variant": v, "n": int(sub["n"]), "x": int(sub["conversions"])})
    return Experiment(name=name, hypothesis=hypothesis, primary_metric=primary, rows=rows)

def build_stats_json(rows: List[Dict[str, Any]], metric_name:str) -> Tuple[Dict[str,Any], ZTestResult]:
    assert len(rows) >= 2, "Need at least two variants (control + treatment)."
    c, t = rows[0], rows[1]
    res = z_test_proportions(c["n"], c["x"], t["n"], t["x"], alpha=0.05)
    payload = {
        "metric": metric_name,
        "baseline": {"variant": rows[0]["variant"], "n": c["n"], "x": c["x"], "rate": res.control_rate},
        "treatment": {"variant": rows[1]["variant"], "n": t["n"], "x": t["x"], "rate": res.treatment_rate},
        "lift": res.lift,
        "p_value": res.p_value,
        "ci": [res.ci_low, res.ci_high]
    }
    return payload, res


In [4]:
# Simple RAG: read rag/past_experiemnts.jsonl + TF-IDF retriever

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def load_past_experiments(jsonl_path: Path) -> List[Dict[str, Any]]:
    if not jsonl_path.exists():
        # bootstrap with a couple of examples if file missing
        samples = [
            {"id":"exp_101","name":"Signup CTA Contrast","date":"2025-07-12","metric":"signup_rate","verdict":"Win",
             "summary":"Higher contrast improved clarity on mobile; negligible effect on desktop.",
             "learnings":["Mobile users responded to contrast","Desktop insensitive to CTA color"],
             "context":"Unauthenticated traffic"},
            {"id":"exp_102","name":"Checkout Short Copy","date":"2025-08-02","metric":"completion_rate","verdict":"Lose",
             "summary":"Shorter copy reduced trust for first-time users.",
             "learnings":["Loss on new users","Better for returning users"],
             "context":"Checkout step 2"}
        ]
        with open(jsonl_path, "w") as f:
            for s in samples: f.write(json.dumps(s)+"\n")
    docs = []
    with open(jsonl_path) as f:
        for line in f:
            try:
                r = json.loads(line)
                body = (
                    f"Name: {r.get('name','')}\nVerdict: {r.get('verdict','')}\n"
                    f"Metric: {r.get('metric','')}\nSummary: {r.get('summary','')}\n"
                    f"Learnings: {', '.join(r.get('learnings', []))}\nContext: {r.get('context','')}"
                )
                docs.append({"id": r.get("id",""), "text": body})
            except Exception:
                continue
    return docs

class TfidfRetriever:
    def __init__(self, texts: List[str]):
        self.vectorizer = TfidfVectorizer(min_df=1, stop_words="english")
        self.matrix = self.vectorizer.fit_transform(texts)
        self.texts = texts

    def top_k(self, query: str, k=5) -> List[Tuple[str, float]]:
        qv = self.vectorizer.transform([query])
        sims = cosine_similarity(qv, self.matrix).ravel()
        idxs = sims.argsort()[::-1][:k]
        return [(self.texts[i], float(sims[i])) for i in idxs]

def build_retriever(jsonl_path=RAGDIR / "past_experiments.jsonl") -> Tuple[TfidfRetriever, List[Dict[str,Any]]]:
    docs = load_past_experiments(jsonl_path)
    texts = [d["text"] for d in docs]
    retriever = TfidfRetriever(texts) if texts else None
    return retriever, docs


In [5]:
# LLM chains (Ollama) with robust JSON parsing + RAG context

from pydantic import BaseModel
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser

# ---- Schemas
class MetricResult(BaseModel):
    metric: str
    baseline_value: float
    treatment_value: float
    lift: float
    p_value: float
    ci: List[float]  # [low, high]

class ExperimentSummary(BaseModel):
    title: str
    verdict: str
    summary: str
    key_metrics: List[MetricResult]
    drivers: List[str]
    caveats: List[str]
    recommended_actions: List[str]

class TestIdea(BaseModel):
    title: str
    hypothesis: str
    rationale: str
    metrics: List[str]
    sample_size_hint: Optional[str] = None
    risks: List[str]
    instrumentation: List[str]

class Suggestions(BaseModel):
    overall_theme: str
    ideas: List[TestIdea]

STRICT = (
    "Return ONLY valid JSON per the schema. No markdown/code fences/prose. "
    "Numbers must be plain JSON numbers (no NaN/Infinity). No extra fields."
)

# ---- Chains
# ---- Chains: return (text_pipe, parser) so we can do safe parsing
def build_summarize_chain(model_name="llama3"):
    llm = OllamaLLM(model=model_name, temperature=0.1)
    parser = PydanticOutputParser(pydantic_object=ExperimentSummary)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are an experimentation analyst. Be precise and actionable. " + STRICT),
        ("human",
         "Experiment: {name}\nHypothesis: {hypothesis}\nPrimary metric: {primary_metric}\n"
         "Stats (json): {stats_json}\nSegments (json): {segments_json}\n\n"
         "Write a structured summary.\n{format_instructions}")
    ]).partial(format_instructions=parser.get_format_instructions())
    text_pipe = prompt | llm      # stop before parser
    return text_pipe, parser

def build_suggestions_chain(model_name="llama3"):
    llm = OllamaLLM(model=model_name, temperature=0.2)
    parser = PydanticOutputParser(pydantic_object=Suggestions)
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Propose practical next tests grounded in outcome and provided RAG snippets. " + STRICT),
        ("human",
         "Heuristics: {heuristics}\nOutcome: {verdict}\nKey learnings: {learnings}\nConstraints: {constraints}\n"
         "Experiment: {name}\nHypothesis: {hypothesis}\n\n"
         "Relevant past learnings (use as evidence, quote sparingly):\n{kb_snippets}\n\n"
         "Propose exactly three ideas with rationale and one instrumentation suggestion.\n{format_instructions}")
    ]).partial(format_instructions=parser.get_format_instructions())
    text_pipe = prompt | llm
    return text_pipe, parser

def verdict_emoji(v: str) -> str:
    return {"Win":"✅", "Lose":"❌", "Inconclusive":"🟨"}.get(v, "ℹ️")


In [6]:
# ---- Safe JSON utilities (no RetryOutputParser needed)
import re, json
from typing import Any, Dict, Tuple

def _strip_code_fences(s: str) -> str:
    # remove ```json ... ``` or ``` ... ```
    return re.sub(r"^```(?:json)?\s*|\s*```$", "", s.strip(), flags=re.I|re.M)

def _extract_json_block(s: str) -> str:
    s = _strip_code_fences(s)
    # find first balanced {...} or [...]
    # simple bracket counter to avoid greedy regex traps
    for open_ch, close_ch in [("{", "}"), ("[", "]")]:
        start = s.find(open_ch)
        if start != -1:
            depth = 0
            for i, ch in enumerate(s[start:], start=start):
                if ch == open_ch: depth += 1
                elif ch == close_ch: depth -= 1
                if depth == 0:
                    return s[start:i+1]
    # fallback: try the first JSON-looking block with regex
    m = re.search(r'(\{.*\}|\[.*\])', s, flags=re.S)
    return m.group(1) if m else s

def run_llm_to_text(pipe, inputs: Dict[str, Any]) -> str:
    """Invoke a (prompt|llm) pipeline and return raw string content."""
    msg = pipe.invoke(inputs)
    # langchain message objects often have .content; fall back to str
    return getattr(msg, "content", str(msg))

def safe_invoke(pipe, parser, inputs: Dict[str, Any], max_retries: int = 2):
    """Run LLM -> raw text -> extract JSON -> parse with Pydantic, with retries & repair."""
    last_err = None
    for attempt in range(max_retries + 1):
        raw = run_llm_to_text(pipe, inputs)
        try:
            # first try as-is
            return parser.parse(raw)
        except Exception as e1:
            # try with code-fence and block extraction
            try:
                cleaned = _extract_json_block(raw)
                return parser.parse(cleaned)
            except Exception as e2:
                last_err = e2
                # small guidance tweak for the next retry
                inputs = dict(inputs)
                inputs["format_instructions"] = parser.get_format_instructions() + \
                    "\nReturn ONLY the JSON object. No prose. No code fences."
                continue
    # bubble up a compact hint for debugging
    raise type(last_err)(f"{last_err}\n--- Raw LLM output snippet ---\n{raw[:800]}")


In [12]:
# Main runner (single experiment) with RAG

# ---- Build retriever once (loads or bootstraps rag/past_experiments.jsonl)
retriever, rag_docs = build_retriever()

def run_one(csv_path: str, model_name="llama3", constraints="Low eng bandwidth next sprint") -> Dict[str, Any]:
    exp = load_experiment_csv(csv_path)
    stats_json, res = build_stats_json(exp.rows, metric_name=exp.primary_metric)
    verdict = verdict_from_result(res)

    # RAG: use experiment name + hypothesis + metric as query
    kb_snips = ""
    if retriever:
        q = f"{exp.name} {exp.hypothesis} {exp.primary_metric}"
        tops = retriever.top_k(q, k=5)
        kb_snips = "\n---\n".join([t for (t,score) in tops])

        sum_pipe, sum_parser = build_summarize_chain(model_name)
        summary = safe_invoke(sum_pipe, sum_parser, {
            "name": exp.name,
            "hypothesis": exp.hypothesis,
            "primary_metric": exp.primary_metric,
            "stats_json": json.dumps(stats_json, default=float),
            "segments_json": "[]"
        })
        summary = summary if isinstance(summary, dict) else summary.dict()
        
        # pick heuristics with a safe default
        DEFAULT_HEURISTICS = """
        - If Win but lift < 5%, recommend cheap iteration (copy/contrast/placement).
        - If Lose, consider rollback, alternative framing, or segmentation.
        - If Inconclusive, extend run or increase sample; consider higher-signal proxy.
        - Always include an instrumentation improvement.
        - Pricing-specific: discount ladders, price anchoring, strike-through formatting, margin guardrails.
        """
        H = globals().get("HEURISTICS", DEFAULT_HEURISTICS)

        sug_pipe, sug_parser = build_suggestions_chain(model_name)
        suggestions = safe_invoke(sug_pipe, sug_parser, {
            "heuristics": H,
            "verdict": verdict,
            "learnings": "; ".join(summary.get("drivers", [])),
            "constraints": constraints,
            "name": exp.name,
            "hypothesis": exp.hypothesis,
            "kb_snippets": kb_snips
        })
        suggestions = suggestions if isinstance(suggestions, dict) else suggestions.dict()


    bundle = {
        "csv": csv_path,
        "experiment": {"name": exp.name, "hypothesis": exp.hypothesis, "primary_metric": exp.primary_metric},
        "stats": stats_json,
        "verdict": verdict,
        "summary": summary,
        "suggestions": suggestions
    }
    # Save outputs
    with open(OUT / "summary.json", "w") as f: json.dump(summary, f, indent=2)
    with open(OUT / "suggestions.json", "w") as f: json.dump(suggestions, f, indent=2)
    return bundle

def format_md(bundle: Dict[str,Any]) -> str:
    s = bundle["summary"]; exp = bundle["experiment"]; v = bundle["verdict"]
    km = (s.get("key_metrics") or [{}])[0]
    ci = km.get("ci", "N/A")
    ci_txt = "[" + ", ".join(f"{x:.6f}" for x in ci) + "]" if isinstance(ci, list) else str(ci)
    actions = s.get("recommended_actions", [])[:3]
    ideas = [i.get("title","") for i in bundle["suggestions"].get("ideas", [])]
    return f"""### {verdict_emoji(v)} {s.get('title', exp['name'])}
**Verdict:** {v}  
**Primary metric:** {exp['primary_metric']}  
**Summary:** {s.get('summary','(no summary)')}

**Key metric:** {km.get('metric','N/A')}
- Control: {km.get('baseline_value','N/A')}
- Treatment: {km.get('treatment_value','N/A')}
- Lift: {km.get('lift','N/A')}
- p-value: {km.get('p_value','N/A')}
- CI: {ci_txt}

**Top Actions:**
- {(actions[0] if len(actions)>0 else '')}
- {(actions[1] if len(actions)>1 else '')}
- {(actions[2] if len(actions)>2 else '')}

**Next Test Ideas:** {", ".join(ideas) if ideas else "(none)"}
"""

# EXAMPLE RUN (change the CSV path as needed)
csv_path = str(DATA / "JP_pricing_test.csv")  # e.g., data/sample_ab_win.csv
bundle = run_one(csv_path, model_name="llama3")
from IPython.display import Markdown, display
display(Markdown(format_md(bundle)))


/var/folders/8s/k2gk5s4s2nv619kgh3w46zyw0000gn/T/ipykernel_8971/634704735.py:26: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  summary = summary if isinstance(summary, dict) else summary.dict()
/var/folders/8s/k2gk5s4s2nv619kgh3w46zyw0000gn/T/ipykernel_8971/634704735.py:48: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  suggestions = suggestions if isinstance(suggestions, dict) else suggestions.dict()


### ✅ Experiment: Japan Pricing Test: actual vs 30% discount
**Verdict:** Win  
**Primary metric:** conversion_rate  
**Summary:** Providing a 30% discount led to a significant increase in conversion rate and ultimately more billings.

**Key metric:** conversion_rate
- Control: 0.0016507841224581677
- Treatment: 0.003986243551664843
- Lift: 0.002335459429206675
- p-value: 0.00035119951427731166
- CI: [0.001055, 0.003616]

**Top Actions:**
- 
- 
- 

**Next Test Ideas:** Copy/Contrast Test, Discount Ladder Test, Price Anchoring Test


In [12]:
# (Optional) Batch: run all CSVs in/data and write per-test markdown

def run_batch(data_dir=DATA, model_name="llama3"):
    reports = OUT / "reports_rag"
    reports.mkdir(exist_ok=True, parents=True)
    results = []
    for p in sorted(Path(data_dir).glob("*.csv")):
        try:
            b = run_one(str(p), model_name=model_name)
            results.append(b)
            with open(reports / f"{p.stem}.md","w") as f:
                f.write(format_md(b))
            print("OK ->", p.name)
        except Exception as e:
            print("FAIL ->", p.name, e)
    with open(OUT / "batch.jsonl","w") as f:
        for r in results: f.write(json.dumps(r)+"\n")
    print("Saved:", OUT / "batch.jsonl")
    return results

# Uncomment to run all:
# batch = run_batch()


In [ ]:
# RAG file format (edit as you like)
# Create/edit rag/past_experiemnts.jsonl (one json per line):

{"id":"exp_201","name":"Pricing Banner Trust Badges","date":"2025-09-01","metric":"checkout_completion","verdict":"Win","summary":"Trust badges increased completion for first-time buyers.","learnings":["First-time users need reassurance","Badges near payment form worked best"],"context":"Checkout step 3"}
{"id":"exp_202","name":"Hero Copy Shortening","date":"2025-09-18","metric":"signup_rate","verdict":"Lose","summary":"Shorter copy reduced clarity for cold traffic.","learnings":["Cold traffic prefers clarity over brevity"],"context":"Landing page"}
